# Cloud AI Platform Prediction - scikit-learn

In [3]:
import json
import os
import pandas as pd
import time

In [4]:
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = "gs://jk-demo-artifacts"

## Create a model resource

In [7]:
MODEL_NAME = "octane_regression"
LABELS = "task=regression,domain=chemometrics"

In [8]:
!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

Created ml engine model [projects/jk-demo1/models/octane_regression].


## Create a model version from a trained model

In [9]:
TRAINED_MODEL_PATH = 'gs://jk-demo-artifacts/models/octane-regressor-r14/'
MODEL_VERSION = 'v01'

In [10]:
!gcloud ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$TRAINED_MODEL_PATH \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

Creating version (this might take a few minutes)......done.                    


### Test the model version
#### Prepare a file with test instances

In [11]:
INPUT_FILE = '../datasets/testing_instances.json'

df_test = pd.read_csv(TESTING_DATA_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [12]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances $INPUT_FILE

[88.59553605661218, 88.84093111257747, 87.35784874687317, 88.73399177287303, 85.26014664670036, 88.12292248697285]
